In [1]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer
from src.config import get_configs

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()
print("hello")

from warnings import simplefilter 
simplefilter(action='ignore', category=DeprecationWarning)

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/envs/registration.py:415: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
pybullet build time: Jun 23 2022 12:25:14


hello


In [3]:

seed = 1
#env_name = "CartPole-v1" ## Action kernel + State_norm looks very well for cartpole
env_name = "Swimmer-v4" ##  State_norm stabilizes training 
#env_name = "Hopper-v2"
#env_name = "Walker2d-v3"
kernel_name = "rbfstate" ## "linearstate" /"rbfstate"

env_config,likelihood_config,kernel_config,mean_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
env_config["manipulate_state"] = False

optimizer_config = {
        "n_eval":2, ## 3 for cartpole (very noisy)
        ### for GIBO
        "n_max":32, 
        "n_info_samples":16,
        "delta":0.1, ## 0.01 for cartpole
        ### hessian normalisation applies only for rbf
        "normalize_gradient": True,
        "standard_deviation_scaling":False,
}


likelihood_config = {
                "noise_hyperprior":gpytorch.priors.torch_priors.UniformPrior(a=0.01,b=0.05),
                "noise_constraint":gpytorch.constraints.constraints.Interval(0.01,0.05)
                }


kernel_config = {
        "use_ard":True,
        "kernel_name":kernel_name,
        #"lengthscale_hyperprior":gpytorch.priors.torch_priors.GammaPrior(2,0.2),
        #"lengthscale_hyperprior":gpytorch.priors.torch_priors.NormalPrior(1,0,),
        "lengthscale_constraint":gpytorch.constraints.constraints.Interval(0.1,10),
        #"outputscale_hyperprior":gpytorch.priors.torch_priors.GammaPrior(2,0.4),
        "outputscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.01),
        }

mean_config = {
                "advantage":False,
        }


policy_config = {
        "add_layer":[],### can be empty or [8,7] for adding 2 layers with width 8,7  neurons respectively
        "add_bias":False,
}

trainer_config = {
        "n_steps":1000, 
        "report_freq":100,
        "save_best":False,
        "wandb_logger":True,
        "run_name" : env_name+kernel_name+str(seed),
        "wandb_config": {**env_config,**optimizer_config,**likelihood_config,**kernel_config,**policy_config}
}


model,objective_env,optimizer = setup_experiment(env_config,mean_config,kernel_config,likelihood_config,policy_config,optimizer_config,
                                seed=seed)

trainer = Trainer(model,objective_env,optimizer,**trainer_config)
rslt= trainer.run()

### ADD LR SCHEDULAR  / FIX DISCRETIZATION ===> ENJOY WEEKEND :DDD

MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
 Gibo will use 32 last points to fit GP and 16 info samples
fixing seed to  1


Mean MAE,▁▁▂▂▁▂▃▅█▇▂▂▃▂
acq_diff,▁▁▁▁▁▁▁▂▂▁▁▁█▃▂▁▅▂▁▁▁▂▁▁▁▁▅▂▁▁▃▂▁▁▁▁▁▁▁▁
acq_value (after finish),▁▁▁▂▁▁▁▆▇█▇█▇▇
action_distance_to_local,▂▁▁▆▆▅▆▁▆▅▅▄▅█▇▅▆▇▅▆▆▅▆▇▆▇▅█▅▆▅█▅▆▅▇▆▆▅▅
covar_lengthscale mean,▁▁▁▁▁▁▃████▁██
covar_lengthscale std,▁▁▁▁▁▁█▅▃▃▄▂▃▃
covar_output_scale,████▁▁▁▁▁▁▁▁▁▁
max_covar(before hessian),▁▁▁█▁▁▂▂▂▃▁▂▂▁
max_grad(before hessian),▂▁▂▂▁▁▅▇▅▅▆█▄▁
max_return,▁▁▂▂▃▃▃▄▆██████
max_var/mean,▁▁▁█▁▁▁▁▁▁▁▁▁▁


In [ ]:
# import scipy.stats as stats 
# import numpy as np
# import matplotlib.pyplot as plt

# alpha = 2
# beta = 0.3
# x = np.linspace (0, 10, 200) 
# y1 = stats.gamma.pdf(x, a=alpha, scale=1/beta)
# plt.plot(x,y1)